<a href="https://colab.research.google.com/github/harshithamadarapu/Team16_Hinglish-Auto-suggestions/blob/main/bigram_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the dataset
dataset_path = '/content/data.xlsx'
df = pd.read_excel(dataset_path)

# Display the first few rows of the dataset
df.head()

,phrases
0,mere timer ko roko
1,Please Liz ke liye video message banaye
2,Alarm create karo at 4 pm and 9 pm on tuesday
3,Library events Cedar Park me hai
4,Please timer ko rokey


In [ ]:
!pip install gradio

In [ ]:
import pandas as pd
from collections import defaultdict
import re
import json
from collections import Counter

# Load the dataset
dataset_path = '/content/data.xlsx'
df = pd.read_excel(dataset_path)

# Clean and preprocess the dataset
def preprocess_dataset(df):
    word_pairs = defaultdict(list)  # Use list to keep track of the word pairs

    # Loop through the phrases and create word pairs
    for phrase in df['phrases']:
        if isinstance(phrase, str):  # Only process valid strings
            phrase = phrase.lower()  # Convert text to lowercase
            # Remove numbers using regex
            phrase = re.sub(r'\d+', '', phrase)
            words = re.findall(r'\w+', phrase)  # Split by word using regex

            for i in range(len(words) - 1):
                word_pairs[words[i]].append(words[i+1])  # Add the next word to the current word

    # For each word, keep the 5 most common word pairs
    top_word_pairs = {}
    for word, pairs in word_pairs.items():
        most_common_pairs = [pair for pair, count in Counter(pairs).most_common(5)]  # Get the top 5 most common pairs
        top_word_pairs[word] = most_common_pairs

    return top_word_pairs

# Preprocess the dataset to get word pairs
top_word_pairs = preprocess_dataset(df)

# Saving the top word pairs to a file for further use
with open('/content/top_word_pairs.json', 'w') as f:
    json.dump(top_word_pairs, f, indent=4)

# Check the structure of the top word pairs
print("Example word pairs for 'timer':", top_word_pairs.get('timer', []))


Example word pairs for 'timer': ['ko', 'set', 'par', 'me', 'shuru']


In [ ]:
import gradio as gr
import json

# Load the word pairs from the file
with open('/content/top_word_pairs.json', 'r') as f:
    top_word_pairs = json.load(f)

# Function to get next word suggestions based on the input phrase
def get_next_word_suggestions(input_phrase):
    # Clean the input phrase (lowercase and split by space)
    input_phrase = input_phrase.lower()
    words = input_phrase.split()

    if not words:
        return "Please enter a word or phrase."

    # Get the last word of the input phrase
    last_word = words[-1]

    # Fetch the next word suggestions based on the last word
    suggestions = top_word_pairs.get(last_word, [])

    if not suggestions:
        return f"No suggestions available for '{last_word}'."

    # Return the top 5 most repeated next words
    return suggestions[:5]

# Create a Gradio interface with live input updates
iface = gr.Interface(
    fn=get_next_word_suggestions,
    inputs=gr.Textbox(placeholder="Type a word or phrase..."),  # Use Textbox for word input
    outputs="text",  # Output will be in text format
    title="Hinglish Real-Time Auto-Suggestions",
    description="Type a word or phrase to get the next word suggestions based on a pre-trained dataset.",
    live=True  # Enable live updates for the entire interface
)

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c42dabd30694148887.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
